In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import random
import pandas as pd
import sqlite3
import re
from datetime import datetime

conn = sqlite3.connect('Iphone.db')

def get_series(model):
    model = str(model).upper()
    if 'SE' in model:
        return 'se'
    elif '17' in model:
        return '17'
    elif '16' in model:
        return '16'
    elif '15' in model:
        return '15'
    elif '14' in model:
        return '14'
    elif '13' in model:
        return '13'
    elif '12' in model:
        return '12'
    elif '11' in model:
        return '11'
    elif 'XS' in model or 'XR' in model or 'X' in model:
        return 'x'
    elif '8' in model:
        return '8'
    else:
        return '其他'

def categorize_iphone_model(model_str):
    if pd.isna(model_str) or model_str.strip() == '':
        return None
    
    model_lower = str(model_str).lower().replace(' ', '')
    
    rules = [
        (['17', 'promax'], '17 Pro Max'),
        (['17', 'pro'], '17 Pro'),
        (['17', 'air'], 'Air'),
        (['17'], '17'),
        (['16', 'promax'], '16 Pro Max'),
        (['16', 'pro'], '16 Pro'),
        (['16', 'plus'], '16 Plus'),
        (['16e'], '16e'),
        (['16'], '16'),
        (['15', 'promax'], '15 Pro Max'),
        (['15', 'pro'], '15 Pro'),
        (['15', 'plus'], '15 Plus'),
        (['15'], '15'),
        (['14', 'promax'], '14 Pro Max'),
        (['14', 'pro'], '14 Pro'),
        (['14', 'plus'], '14 Plus'),
        (['14'], '14'),
        (['13', 'promax'], '13 Pro Max'),
        (['13', 'pro'], '13 Pro'),
        (['13', 'mini'], '13 Mini'),
        (['13'], '13'),
        (['12', 'promax'], '12 Pro Max'),
        (['12', 'pro'], '12 Pro'),
        (['12', 'mini'], '12 Mini'),
        (['12'], '12'),
        (['11', 'promax'], '11 Pro Max'),
        (['11', 'pro'], '11 Pro'),
        (['11'], '11'),
        (['xsmax'], 'XS Max'),
        (['xs'], 'XS'),
        (['xr'], 'XR'),
        (['x'], 'X'),
        (['8', 'plus'], '8 Plus'),
        (['8'], '8'),
        (['se'], 'SE'),
    ]
    
    for keywords, category in rules:
        if all(keyword in model_lower for keyword in keywords):
            return category
    
    return None

def classify_appearance(x):
    if '優' in x:
        return '優'
    elif x == '正常':
        return '好'
    else:
        return x

def process_warranty(text):
    text = str(text)
    
    if '三個月' in text or '3個月' in text:
        return 'three month'
    
    date_pattern_chinese = r'(\d+)年(\d+)月(\d+)日'
    match = re.search(date_pattern_chinese, text)
    
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        day = int(match.group(3))
        
        if year < 1911:
            year = year + 1911
        
        return f"{year}/{month:02d}/{day:02d}"
    
    date_pattern = r'(\d+)[./](\d+)[./](\d+)'
    match = re.search(date_pattern, text)
    
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        day = int(match.group(3))
        
        if year < 1911:
            year = year + 1911
        
        return f"{year}/{month:02d}/{day:02d}"
    
    text = re.sub(r'[\u4e00-\u9fff]+', '', text)
    text = text.strip()
    
    return text

def extract_product_code(text):
    """從內部文字中提取編號，例如: 'iPhone 14 128G 紫色 (R679-0717)' -> 'R679-0717'"""
    match = re.search(r'\(([A-Z]\d+-\d+)\)', text)
    if match:
        return match.group(1)
    return None

def get_existing_product_codes(conn):
    """從資料庫取得現有的所有編號"""
    try:
        query = "SELECT 編號 FROM GS_iphone WHERE 狀態 = '上架中'"
        df = pd.read_sql(query, conn)
        return set(df['編號'].tolist())
    except:
        return set()

def collect_all_product_codes(driver, series_list):
    """階段一：快速收集所有列表頁的商品編號和網址"""
    print("\n" + "="*60)
    print("階段一：快速掃描所有商品編號")
    print("="*60)
    
    all_products = {}  # {編號: 網址}
    
    for series in series_list:
        page = 1
        series_count = 0
        
        while True:
            list_url = f"https://www.guardstation.com.tw/categories/second-hand-iphone-{series}-series?page={page}&sort_by=&order_by=&limit=72"
            driver.get(list_url)
            print(f"iPhone {series} 系列: 第 {page} 頁", end=" ")
            time.sleep(random.uniform(1, 1.5))

            current_url = driver.current_url
            if "guardstation.com.tw/" == current_url.rstrip('/').split('/')[-1] or "/categories/" not in current_url:
                print("→ 系列已售罄或不存在")
                break

            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="/products/iphone"]'))
                )
            except:
                print("→ 沒有商品")
                break
            
            # 收集本頁所有商品的編號和網址
            products = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/products/iphone"]')
            page_count = 0
            
            for p in products:
                text = p.text  # 改用 text 而非 title
                url = p.get_attribute('href')
                
                if text:
                    code = extract_product_code(text)
                    if code and code not in all_products:
                        all_products[code] = url
                        page_count += 1
            
            series_count += page_count
            print(f"→ 找到 {page_count} 個商品")
            
            if not products:
                break
            
            page += 1
        
        print(f"iPhone {series} 系列完成！共 {series_count} 個商品\n")
    
    print(f"總計找到 {len(all_products)} 個商品")
    return all_products

def scrape_product_detail(driver, url, code):
    """爬取單一商品的詳細資料"""
    try:
        driver.get(url)
        time.sleep(random.uniform(1.5, 2))
        
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "Product-summary"))
        )
        
        data = {
            "編號": code,
            "機型": "",
            "容量": "",
            "顏色": "",
            "保固": "",
            "外觀": "",
            "功能": "",
            "分店位置": "",
            "優惠價格": "",
            "原價格": "",
            "抓取月份": datetime.now().strftime("%Y-%m"),
            "狀態": "上架中"
        }
        
        summary_block = driver.find_element(By.CLASS_NAME, "Product-summary")
        lines = summary_block.text.split('\n')
        
        for line in lines:
            if "機型" in line:
                data["機型"] = line.split("：")[-1].strip()
            elif "容量" in line:
                data["容量"] = line.split("：")[-1].strip()
            elif "顏色" in line:
                data["顏色"] = line.split("：")[-1].strip()
            elif "保固" in line:
                data["保固"] = line.split("：")[-1].strip()
            elif "外觀" in line:
                data["外觀"] = line.split("：")[-1].strip()
            elif "功能" in line:
                data["功能"] = line.split("：")[-1].strip()
            elif "分店位置" in line or "店位置" in line:
                data["分店位置"] = line.split("：")[-1].strip()
        
        try:
            sale_price = driver.find_element(By.CSS_SELECTOR, ".price-sale").text
            data["優惠價格"] = sale_price.replace("NT$", "").replace(",", "").strip()
        except:
            data["優惠價格"] = ""
        
        try:
            regular_price = driver.find_element(By.CSS_SELECTOR, ".price-regular.price-crossed").text
            data["原價格"] = regular_price.replace("NT$", "").replace(",", "").strip()
        except:
            try:
                regular_price = driver.find_element(By.CSS_SELECTOR, ".price-regular.price").text
                data["原價格"] = regular_price.replace("NT$", "").replace(",", "").strip()
            except:
                data["原價格"] = ""
        
        return data
        
    except Exception as e:
        print(f"處理商品 {code} 時發生錯誤: {e}")
        return None

def mark_delisted_products(conn, current_codes):
    """標記已下架的商品"""
    print("\n" + "="*60)
    print("階段四：標記下架商品")
    print("="*60)
    
    try:
        existing_df = pd.read_sql("SELECT 編號 FROM GS_iphone WHERE 狀態 = '上架中'", conn)
        existing_codes = set(existing_df['編號'].tolist())
        
        delisted_codes = existing_codes - current_codes
        
        if delisted_codes:
            codes_str = "','".join(delisted_codes)
            update_query = f"UPDATE GS_iphone SET 狀態 = '已下架' WHERE 編號 IN ('{codes_str}')"
            conn.execute(update_query)
            conn.commit()
            print(f"已標記 {len(delisted_codes)} 個商品為「已下架」")
            print(f"下架商品編號: {', '.join(list(delisted_codes)[:10])}{'...' if len(delisted_codes) > 10 else ''}")
        else:
            print("沒有商品下架")
            
    except Exception as e:
        print(f"標記下架商品時發生錯誤: {e}")

if __name__ == "__main__":
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=chrome_options)
    # ["se", 8, "x", 11, 12, 13, 14, 15, 16, 17]
    try:
        series_list =["se", 8,"x", 11, 12, 13, 14, 15, 16, 17]
        
        # 階段一：收集所有商品編號
        all_products = collect_all_product_codes(driver, series_list)
        
        # 階段二：比對資料庫，找出新商品
        print("\n" + "="*60)
        print("階段二：比對資料庫")
        print("="*60)
        
        existing_codes = get_existing_product_codes(conn)
        new_codes = set(all_products.keys()) - existing_codes
        
        print(f"資料庫現有商品: {len(existing_codes)} 個")
        print(f"網站目前商品: {len(all_products)} 個")
        print(f"需要新增商品: {len(new_codes)} 個")
        
        # 階段三：只爬取新商品
        if new_codes:
            print("\n" + "="*60)
            print("階段三：爬取新商品詳細資料")
            print("="*60)
            
            new_data = []
            for idx, code in enumerate(new_codes, 1):
                print(f"處理新商品 {idx}/{len(new_codes)}: {code}")
                url = all_products[code]
                data = scrape_product_detail(driver, url, code)
                if data:
                    new_data.append(data)
            
            # 資料處理
            if new_data:
                df = pd.DataFrame(new_data)
                df['系列'] = df['機型'].apply(get_series)
                df['機型'] = df['機型'].str.replace('iPhone ', '')
                df['機型'] = df['機型'].apply(categorize_iphone_model)
                df['外觀'] = df['外觀'].apply(classify_appearance)
                df['保固'] = df['保固'].apply(process_warranty)
                df['保固'] = df['保固'].apply(lambda x: "no warranty" if x == "" else x)
                df['保固_類型'] = df['保固'].apply(lambda x: '店保' if x == 'three month' else ('no warranty' if x == 'no warranty' else '原廠保固'))

                df['價格'] = df['優惠價格'].fillna(df['原價格'])
                df["原價格"] = pd.to_numeric(df["原價格"], errors='coerce')
                df["優惠價格"] = pd.to_numeric(df["優惠價格"], errors='coerce')
                
                # 存入資料庫（使用 append 模式）
                df.to_sql("GS_iphone", con=conn, if_exists="append", index=False)
                print(f"\n成功新增 {len(df)} 筆商品到資料庫")
                
                # 儲存 CSV
                df.to_csv("guardstation_iphone.csv.csv", index=False, encoding="utf-8-sig")
                print("已儲存至 guardstation_iphone.csv.csv")
        else:
            print("\n沒有新商品需要爬取")
        
        # 階段四：標記下架商品
        mark_delisted_products(conn, set(all_products.keys()))
        
        print("\n" + "="*60)
        print("完成！")
        print("="*60)
        
    except Exception as e:
        print(f"發生錯誤: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        conn.close()
        driver.quit()


階段一：快速掃描所有商品編號
iPhone se 系列: 第 1 頁 → 找到 3 個商品
iPhone se 系列: 第 2 頁 → 沒有商品
iPhone se 系列完成！共 3 個商品

iPhone 8 系列: 第 1 頁 → 系列已售罄或不存在
iPhone 8 系列完成！共 0 個商品

iPhone x 系列: 第 1 頁 → 找到 50 個商品
iPhone x 系列: 第 2 頁 → 沒有商品
iPhone x 系列完成！共 50 個商品

iPhone 11 系列: 第 1 頁 → 找到 64 個商品
iPhone 11 系列: 第 2 頁 → 沒有商品
iPhone 11 系列完成！共 64 個商品

iPhone 12 系列: 第 1 頁 → 找到 69 個商品
iPhone 12 系列: 第 2 頁 → 找到 65 個商品
iPhone 12 系列: 第 3 頁 → 找到 72 個商品


KeyboardInterrupt: 